# Anuarios estadísticos de la DGT - Datos estructurados en PDF

Este notebook extrae los datos de matriculación de vehículos por tipo de combustible del anuario estadístico de la DGT (formato PDF) y lo transforma en un formato limpio y estructurado para su posterior analisis.

La información publicada por la DGT que es accesible al público general está publicada en formato PDF, por lo cual, antes de empezar el análisis, tenemos que transformar la información a un formato que nos permita poder trabajar en ella.

## Fuente de datos:
- Dirección General de Tráfico (DGT)

In [1]:
# Importacón de librerias
import os
import pandas as pd
import numpy as np
import tabula

In [2]:
# Directorios y archivos
DIR_BASE = os.getcwd()
DIR_PDF = os.path.join(DIR_BASE, "ANUARIOS DGT")
DIR_OUTPUT = os.path.join(DIR_BASE, "output")

os.makedirs(DIR_OUTPUT, exist_ok=True)

pdf_files = sorted([
    f for f in os.listdir(DIR_PDF) 
    if f.lower().endswith(".pdf")
])

pdf_files

['Anuario-Estadistico-General-2024.pdf']

In [3]:
# Exploración de un ejemplo
muestra_pdf = os.path.join(DIR_PDF, pdf_files[0]) 
print(muestra_pdf)

dfs_muestra = tabula.read_pdf(
    muestra_pdf,
    pages=18,
    multiple_tables=False,
    guess=False
)

dfs_muestra = dfs_muestra[0]
print(dfs_muestra.head())

Failed to import jpype dependencies. Fallback to subprocess.
No module named 'jpype'


c:\Users\controllergamboa2\Desktop\INFORMES POWER BI\P1\ANUARIOS DGT\Anuario-Estadistico-General-2024.pdf
                    Anuario Estadístico General 2024 Unnamed: 1
0     Tabla 6.- MATRICULACIÓN DE VEHÍCULOS DE GASOIL        NaN
1  TractoresAños Camiones y TotalFurgonetas Autob...        NaN
2                 1990 188.475 2.829 132.346 0 8.894  0 332.544
3                 1991 171.041 2.937 114.264 0 7.097  0 295.339
4                 1992 172.109 2.775 163.676 0 5.630  0 344.190


Al extrarer los datos de una muestra del anuario de la DGT, observamos que la estructura no es correcta. Para poder proceder, tenemos que asegurarnos que los datos están estructurados.

A continuación se detallan los pasos a seguir para la limpieza y extracción de la base de datos:

In [4]:
# Paso 1: Se seleccionan únicamente las filas que contienen datos
df_raw_muestra = dfs_muestra.iloc[2:37,0]

# Paso 2: Se dividen las columnas a partir de los espacios en blanco
df_split_muestra = df_raw_muestra.str.split(r"\s+", expand=True)

# Paso 3: Se le asigna nombre a las columnas
df_split_muestra.columns = [
    "año",
    "camiones_furgonetas",
    "autobuses",
    "turismos",
    "motocicletas",
    "tractores_industriales",
    "otros_vehiculos"
]

# Paso 4: Se realizan las transformaciones necesarias y se convierten las columnas numéricas a tipo entero   
df_split_muestra["año"] = df_split_muestra["año"].astype(int)

num_cols = df_split_muestra.columns.drop("año")

df_split_muestra[num_cols] = df_split_muestra[num_cols].fillna(0)

for col in num_cols:
    df_split_muestra[col] = (
        df_split_muestra[col]
        .str.replace(".", "", regex=False)
    )

#Paso 5: Se muestran los resultados
print(df_split_muestra.head())

    año camiones_furgonetas autobuses turismos motocicletas  \
2  1990              188475      2829   132346            0   
3  1991              171041      2937   114264            0   
4  1992              172109      2775   163676            0   
5  1993              121458      1915   167316            0   
6  1994              140737      1853   244732            0   

  tractores_industriales otros_vehiculos  
2                   8894             NaN  
3                   7097             NaN  
4                   5630             NaN  
5                   3077             NaN  
6                   4928             NaN  


Una vez validada la transformación sobre una muestra representativa, se desarrolló una función reutilizable que encapsula todo el proceso de limpieza y estructuración de los datos. Este enfoque permite aplicar de manera consistente la misma lógica a todas las páginas necesarias para el análisis.

Para este análisis, es necesario extraer los datos de matriculación de vehículos correspondientes a las páginas 17, 18 y 19 del documento PDF. Adicionalmente, se incorpora una columna que clasifica los registros según el tipo de combustible. Dicha clasificación se realiza asignando **“Gasolina”** a los registros de la página 17, **“Gasoil”** a los de la página 18 y **“Otros combustibles”** a los de la página 19.

In [5]:
def extract_fuel_table(pdf_path, page, fuel_name):

    dfs = tabula.read_pdf(
        pdf_path,
        pages=page,
        multiple_tables=False,
        guess=False
    )

    df = dfs[0]

    # Seleccionamos solo filas con datos
    df_raw = df.iloc[2:37, 0]

    # Split por espacios variables
    df_split = df_raw.str.split(r"\s+", expand=True)

    # Asignar nombres de columnas
    df_split.columns = [
        "año",
        "camiones_furgonetas",
        "autobuses",
        "turismos",
        "motocicletas",
        "tractores_industriales",
        "otros_vehiculos"
    ]

    # Tipos
    df_split["año"] = df_split["año"].astype(int)

    num_cols = df_split.columns.drop("año")

    df_split[num_cols] = df_split_muestra[num_cols].fillna(0)

    for col in num_cols:
        df_split[col] = (
            df_split[col]
            .str.replace(".", "", regex=False)
        )

    # Añadir columna combustible
    df_split["combustible"] = fuel_name

    return df_split

In [9]:
pdf = os.path.join(DIR_PDF, pdf_files[0])

df_gasolina = extract_fuel_table(pdf, 17, "GASOLINA")
df_gasoil = extract_fuel_table(pdf, 18, "GASOIL")
df_otros = extract_fuel_table(pdf, 19, "OTROS COMBUSTIBLES")

df_final = pd.concat(
    [df_gasolina, df_gasoil, df_otros],
    ignore_index=True
)

print(df_final.head())

    año camiones_furgonetas autobuses turismos motocicletas  \
0  1990              188475      2829   132346            0   
1  1991              171041      2937   114264            0   
2  1992              172109      2775   163676            0   
3  1993              121458      1915   167316            0   
4  1994              140737      1853   244732            0   

  tractores_industriales otros_vehiculos combustible  
0                   8894             NaN    GASOLINA  
1                   7097             NaN    GASOLINA  
2                   5630             NaN    GASOLINA  
3                   3077             NaN    GASOLINA  
4                   4928             NaN    GASOLINA  


Para finalizar, guardamos la base de datos resultante en un archivo csv para su porterior uso.

In [ ]:
df_final.to_csv(f"{DIR_OUTPUT}/df_final.csv", index=False)